<a href="https://colab.research.google.com/github/realyou0312/Bigdata-Gisa/blob/main/JY_1_Example_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier

In [ ]:
path = '/content/drive/MyDrive/빅분기 실기/예제문제/'
df = pd.read_csv(path + 'mtcars.csv',index_col=0)

## 1. mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 최소 최대 척도(min-max scale)로 변환한 후 0.5보다 큰 값을 가지는 레코드 수를 구하시오.

In [ ]:
scaler = MinMaxScaler()
train = df['qsec'].values.reshape(-1,1)
minmax = scaler.fit_transform(train)
np.count_nonzero(minmax > 0.5)

In [ ]:
temp = df['qsec']
minmax = (temp - temp.min(axis=0)) / (temp.max(axis=0) - temp.min(axis=0))

num = len(minmax[minmax>0.5])
num

## 2. 분석문제

In [ ]:
y_train = pd.read_csv(path + 'y_train.csv')
X_train = pd.read_csv(path + 'X_train.csv',encoding='euc-kr')
test  = pd.read_csv(path + 'X_test.csv',encoding='euc-kr')

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
train = pd.merge(y_train,X_train) # 공통 열 묶임

In [ ]:
train

## Null 처리

In [ ]:
train.isnull().sum()

In [ ]:
train[train['환불금액'].isnull()].gender.value_counts()

In [ ]:
#환불금액 null -> 환불금액 없다는 뜻 -> 0 대체
train['환불금액'] =train['환불금액'].fillna(0)
test['환불금액'] = test['환불금액'].fillna(0)

In [ ]:
 train['주구매상품'].value_counts()

## 이상치 확인 -> 그냥해봄

In [ ]:
numerical_feats = ['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']

In [ ]:
t= train.copy()

In [ ]:
for v in numerical_feats:
    df = t[[v,'gender']]
    
    male = df[df.gender==1]
    female = df[df.gender==0]
    fig, axes = plt.subplots(1,2,figsize=(15,5))
    
    print(v)
    axes[0].hist(male[v],alpha=0.3,color='blue',label='male',bins=40)
    axes[0].hist(female[v],alpha=0.3,color='red',label='female',bins=40)
    axes[0].legend()
    axes[1].boxplot(female[v],positions=[0],widths=0.5)
    axes[1].boxplot(male[v],positions=[1],widths=0.5)
    axes[1].set_xticklabels(['women','men'])
    plt.show()

## Scale / get-dummy / smote / RF Modeling / GSCV

In [ ]:
x = train.drop(['cust_id','gender'],axis=1)
y = train.gender

In [ ]:
x_dum = pd.get_dummies(x)
test_dum = pd.get_dummies(test)

missing_cols = set( x_dum.columns ) - set( test_dum.columns )
for c in missing_cols:
    test_dum[c] = 0
test_dum = test_dum[x_dum.columns]


In [ ]:
scaler = StandardScaler()
x_dum = scaler.fit_transform(x_dum)
test_dum = scaler.transform(test_dum)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_dum, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
smote = SMOTE(random_state=42)
X_train_over,y_train_over = smote.fit_resample(X_train,y_train)

In [ ]:
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X_train_over, y_train_over)

y_pred = rf.predict(X_test)
report =classification_report(y_test, y_pred, target_names=['class 0', 'class 1'])
print(report)

In [ ]:
lr_probs = rf.predict_proba(X_test)
lr_auc = roc_auc_score(y_test, lr_probs[:,1])
lr_auc

In [ ]:
skfold = StratifiedKFold(n_splits=10)

RF = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRF = GridSearchCV(RF, param_grid = rf_param_grid, cv=skfold, scoring="accuracy", n_jobs= 4, verbose = 1)
gsRF.fit(x_dum,y)
RF_best = gsRF.best_estimator_

gsRF.best_score_

In [ ]:
test_gender = pd.Series(gsRF.predict_proba(test_dum)[:,1], name="gender")
results= pd.concat([test.cust_id,test_gender],axis=1)
results.head(3)

In [ ]:
results.to_csv("submission.csv",index=False)